In [1]:
import os

In [2]:
os.chdir(r'D:\data science and AI\2. MLOPS COURSE BY KRISH\14. First Project\byme\datascineceproject')


In [ ]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha=params.alpha,
            l1_ratio= params.l1_ratio,
            target_column=schema.name

        )

        return model_trainer_config

In [ ]:
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train=pd.read_csv(self.config.train_data_path)
        test=pd.read_csv(self.config.test_data_path)

        train_x=train.drop([self.config.target_column],axis=1)
        test_x=test.drop([self.config.target_column],axis=1)
        train_y=train[[self.config.target_column]]
        test_y=test[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir, self.config.model_name))





In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e